In [57]:
import pandas as pd
import json
import websocket

endpoint = 'wss://stream.binance.com:9443/ws'
our_msg = json.dumps({'method':'SUBSCRIBE',
                     'params':['btcusdt@ticker'],'id':1})
scale = 300

In [63]:
df = pd.DataFrame()
signals = pd.DataFrame()
in_position = False
buy_orders,sell_orders = [], []

def on_open(ws):
    ws.send(our_msg)

def on_message(ws,message):
    global df, scale, in_position, buy_orders, sell_orders, signals
    out=json.loads(message)
    out = pd.DataFrame({'price':float(out['c'])}, index=[pd.to_datetime(out['E'],unit='ms')])
    df = pd.concat([df,out],axis=0)

    df = df.tail(scale)
    last_price = df.tail(1).price.values[0]
    # sma_300 = df.price.rolling(scale).mean().tail(1).values[0]

    short_window = 30
    long_window = 150
    stop_loss_percent = 0.002  # 0.5% stop-loss
    signals = sma_crossover_with_stop_loss(df, short_window, long_window, stop_loss_percent, stop_loss_percent)
    last_buy = 0
    buy_signal = False
    sell_signal = False
    stop_loss = 0
    profit_margin = 0
    if len(buy_orders) > 0:
        last_buy = buy_orders[len(buy_orders)-1]

    if signals.shape[0] > long_window:
        buy_signal = signals.tail(1).Signal.values[0] == 1 and signals.tail(2).Signal.values[0] == -1
        if last_buy != 0:
            stop_loss = last_buy * (1 - stop_loss_percent)
            profit_margin = last_buy * (1 + stop_loss_percent)
            sell_signal = signals.tail(1).Price.values[0] < stop_loss or signals.tail(1).Price.values[0] >= profit_margin

    
    if in_position and sell_signal:
        print('sold for ' + str(last_price))
        sell_orders.append(last_price)
        in_position = False
        sell_signal = False
        buy_signal = False
    
    if not in_position and buy_signal:
        print('bounght for ' + str(last_price))
        buy_orders.append(last_price)
        current_buy = buy_orders[len(buy_orders)-1]
        stop_loss_current = current_buy * (1 - stop_loss_percent)
        profit_margin_current = current_buy * (1 + stop_loss_percent)
        print('stop_loss: ' + str(stop_loss_current))
        print('profit_margin: ' + str(profit_margin_current))
        in_position = True
        buy_signal = False
        sell_signal = False

In [51]:
print(signals.shape[0])

120


In [59]:
def simple_moving_average(data, window):
    return data.rolling(window=window).mean()

def sma_crossover_with_stop_loss(data, short_window, long_window, stop_loss_percent, profit_margin_percent):
    signals = pd.DataFrame(index=data.index)
    signals['Price'] = data
    signals['Short_SMA'] = simple_moving_average(data, short_window)
    signals['Long_SMA'] = simple_moving_average(data, long_window)
    signals['Signal'] = 0  # 1: buy, -1: sell
    signals['Stop_Loss'] = 0
    signals['Profit_Margin'] = 0

    # Generate buy signals
    signals.loc[signals['Short_SMA'] > signals['Long_SMA'], 'Signal'] = 1

    # Generate sell signals
    signals.loc[signals['Short_SMA'] <= signals['Long_SMA'], 'Signal'] = -1

    # Calculate stop-loss levels
    for i in range(1, len(signals)):
        if signals.iloc[i - 1]['Signal'] == 1:
            stop_loss_price = (1 - stop_loss_percent) * signals.iloc[i - 1]['Price']
            signals.at[signals.index[i], 'Stop_Loss'] = stop_loss_price
        if signals.iloc[i - 1]['Signal'] == -1:
            profit_margin_price = (1 + profit_margin_percent) * signals.iloc[i - 1]['Price']
            signals.at[signals.index[i], 'Profit_Margin'] = profit_margin_price

    return signals

In [ ]:
ws = websocket.WebSocketApp(endpoint, on_message=on_message,on_open=on_open)
ws.run_forever()

bounght for 29837.99
stop_loss: 29778.31402
profit_margin: 29897.66598


In [42]:
display(signals)
display(buy_orders)
display(sell_orders)

,Price,Short_SMA,Long_SMA,Signal,Stop_Loss,Profit_Margin
2023-07-21 13:40:43.484,29867.90,NaN,NaN,0,0,0
2023-07-21 13:40:44.503,29867.89,NaN,NaN,0,0,0
2023-07-21 13:40:45.607,29870.00,NaN,NaN,0,0,0
2023-07-21 13:40:46.615,29869.99,NaN,NaN,0,0,0
2023-07-21 13:40:47.615,29869.99,NaN,NaN,0,0,0
...,...,...,...,...,...,...
2023-07-21 13:42:38.696,29863.10,29868.740667,NaN,0,0,0
2023-07-21 13:42:39.575,29863.11,29868.751333,NaN,0,0,0
2023-07-21 13:42:40.582,29863.10,29868.735333,NaN,0,0,0
2023-07-21 13:42:41.626,29863.10,29868.719333,NaN,0,0,0


[]

[]

In [4]:
# Sample price data
price_data = pd.Series([100, 110, 120, 130, 140, 150, 160, 170, 160, 150, 140, 130, 120, 110, 100])

# Define parameters
short_window = 5
long_window = 10
stop_loss_percent = 0.05  # 5% stop-loss

# Generate signals
signals = sma_crossover_with_stop_loss(price_data, short_window, long_window, stop_loss_percent)

print(signals)

    Price  Short_SMA  Long_SMA  Signal  Stop_Loss
0     100        NaN       NaN       0        0.0
1     110        NaN       NaN       0        0.0
2     120        NaN       NaN       0        0.0
3     130        NaN       NaN       0        0.0
4     140      120.0       NaN       0        0.0
5     150      130.0       NaN       0        0.0
6     160      140.0       NaN       0        0.0
7     170      150.0       NaN       0        0.0
8     160      156.0       NaN       0        0.0
9     150      158.0     139.0       1        0.0
10    140      156.0     143.0       1      142.5
11    130      150.0     145.0       1      133.0
12    120      140.0     145.0      -1      123.5
13    110      130.0     143.0      -1        0.0
14    100      120.0     139.0      -1        0.0
